###  Planet Analytics API Tutorial

# Getting Analytic Feed Results
This notebook shows how to paginate through Planet Analytic Feed Results for an existing analytics [Subscription](https://developers.planet.com/docs/analytics/#subscriptions) to construct a combined [GeoJSON](https://geojson.org/) feature collection that can be imported into geospatial analysis tools.

You can open this notebook in Colab below:

<a target="_blank" href="https://colab.research.google.com/github/planetlabs/notebooks/blob/master/jupyter-notebooks/analytics/quickstart/02_fetching_feed_results.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup 

#### Import Packages

In [17]:
import os
import requests
import pandas as pd
import json

from dateutil.parser import parse
from datetime import timedelta
from IPython.display import FileLink, FileLinks
import geopandas as gpd

To use this notebook, you need an [API key](https://developers.planet.com/quickstart/apis/) for a Planet account with access to the [Analytics API](https://developers.planet.com/docs/analytics/).
#### API Key and Test Connection
Set `API_KEY` below if it is not already in your notebook as an environment variable.
See the [Analytics API Docs](https://developers.planet.com/docs/analytics/) for more details on authentication.

In [18]:
# If your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.environ.get('PL_API_KEY', 'PASTE_YOUR_KEY_HERE')

# Alternatively, you can just set your API key directly as a string variable:
# API_KEY = "YOUR_PLANET_API_KEY_HERE"

# Construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')
BASE_URL = "https://api.planet.com/analytics/"

subscriptions_list_url = BASE_URL + 'subscriptions' + '?limit=1000'
resp = requests.get(subscriptions_list_url, auth=BASIC_AUTH)
if resp.status_code == 200:
    print('Yay, you can access the Analytics API')
    subscriptions = resp.json()['data']
    print('Available subscriptions:', len(subscriptions))
else:
    print('Something is wrong:', resp.content)

Yay, you can access the Analytics API
Available subscriptions: 561


#### Specify Analytics Subscription of Interest
Below we will list your available subscription ids and some metadata in a dataframe and then select a subscription of interest.

In [19]:
pd.options.display.max_rows = 1000
df = pd.DataFrame(subscriptions)
df['start'] = pd.to_datetime(df['startTime']).dt.date
df['end'] = pd.to_datetime(df['endTime']).dt.date
df[['id', 'title', 'description', 'start', 'end']]

,id,title,description,start,end
0,d113c5ac-6d64-4f7b-b501-16fa45a49293,Weekly road detections over Nairobi,https://hello.planet.com/jira/browse/AN-6006,2022-01-01,2023-06-15
1,ea0b8c2b-7504-4018-bed3-2ce089dd77c5,Weekly building detections over Nairobi,https://hello.planet.com/jira/browse/AN-6006,2022-01-01,2023-06-15
2,a7011f4c-20ec-4344-95fa-24708bee57e1,Weekly road change detection over Nairobi,https://hello.planet.com/jira/browse/AN-6006,2022-01-01,2023-06-15
3,7dd2c0af-d88e-471d-bc76-9ffdb931ed6a,Weekly building change detection over Nairobi,https://hello.planet.com/jira/browse/AN-6006,2022-01-01,2023-06-15
4,520ca04e-cef3-4da7-8187-165059a57ff1,San Diego Area,Ship Detection (50m+),2023-01-01,2023-04-01
5,9ad3f8ac-d8f5-47db-8295-096097d2f6a7,"Spokane, WA",Weekly Road Change Detection,2023-06-01,2024-07-31
6,9a53b5f4-f89e-4de4-9c7a-1e01fa3dc462,"Spokane, WA",Weekly Building Change Detection,2023-06-01,2024-07-31
7,508b6844-5933-4810-a581-0cb18b37ce8b,"Spokane, WA",Weekly Road Detection,2023-06-01,2024-07-31
8,d0dda157-6d3b-4120-99a7-a9dd9fce0d1a,"Spokane, WA",Weekly Building Detection,2023-06-01,2024-07-31
9,42725e4d-9327-4418-9207-5535ba76265b,Road change detection over Zanzibar,https://hello.planet.com/jira/browse/AN-5994,2022-06-01,2023-06-07


Pick a subscription from which to pull results, and replace the ID below.

In [20]:
# This example ID is for a subscription of ship detections in the Port of Oakland
# You can replace this ID with your own subscription ID
SUBSCRIPTION_ID = '9db92275-1d89-4d3b-a0b6-68abd2e94142'

## Getting subscription results
In this section, we will make sure that we can get data from the subscription of interest by fetching the latest page of results.

In [21]:
# Construct the url for the subscription's results collection
subscription_results_url = BASE_URL + 'collections/' + SUBSCRIPTION_ID + '/items'
print("Request URL: {}".format(subscription_results_url))

# Get subscription results collection
resp = requests.get(subscription_results_url, auth=BASIC_AUTH)
if resp.status_code == 200:
    print('Yay, you can access analytic feed results!')
    subscription_results = resp.json()
    print(json.dumps(subscription_results, sort_keys=True, indent=4))
else:
    print('Something is wrong:', resp.content)

Request URL: https://api.planet.com/analytics/collections/9db92275-1d89-4d3b-a0b6-68abd2e94142/items
Yay, you can access analytic feed results!
{
    "features": [
        {
            "created": "2022-10-27T22:53:29.202Z",
            "geometry": {
                "coordinates": [
                    [
                        [
                            -122.321902511231,
                            37.7985556136176
                        ],
                        [
                            -122.321911880908,
                            37.797530647137
                        ],
                        [
                            -122.324780667438,
                            37.797547124339
                        ],
                        [
                            -122.324771337394,
                            37.7985720914244
                        ],
                        [
                            -122.321902511231,
                            37.798555613617

## Pagination

The response JSON above will only include the most recent 250 detections by default. For subscriptions with many results, you can page through 

In [22]:
print(len(subscription_results['features']))

250


More results can be fetched by following the `next` link. Let's look at the links section of the response:

In [23]:
subscription_results['links']

[{'href': 'https://api.planet.com/analytics/collections/9db92275-1d89-4d3b-a0b6-68abd2e94142/items',
  'rel': 'self',
  'type': 'application/geo+json'},
 {'href': 'https://api.planet.com/analytics/collections/9db92275-1d89-4d3b-a0b6-68abd2e94142',
  'rel': 'collection',
  'type': 'application/json'},
 {'href': 'https://api.planet.com/analytics/collections/9db92275-1d89-4d3b-a0b6-68abd2e94142/items?before=06c7f0f0-7d24-4ac2-880d-b2bb3ec70693',
  'rel': 'next',
  'type': 'application/json'}]

To get more results, we will want the link with a `rel` of `next`

In [24]:
def get_next_link(results_json):
    """Given a response json from one page of subscription results, 
    get the url for the next page of results.
    
    Args:
        results_json (dict): The response JSON containing the subscription results.

    Returns:
        str or None: The URL for the next page of results if available, None otherwise.
    """
    for link in results_json['links']:
        if link['rel'] == 'next':
            return link['href']
    return None

In [25]:
next_link = get_next_link(subscription_results)
print('next page url: ' + next_link)

next page url: https://api.planet.com/analytics/collections/9db92275-1d89-4d3b-a0b6-68abd2e94142/items?before=06c7f0f0-7d24-4ac2-880d-b2bb3ec70693


Using this url, we can fetch the next page of results

In [26]:
next_results = requests.get(next_link, auth=BASIC_AUTH).json()
print(json.dumps(next_results, sort_keys=True, indent=4))

{
    "features": [
        {
            "created": "2021-03-11T11:54:55.587Z",
            "geometry": {
                "coordinates": [
                    [
                        [
                            -122.339000091352,
                            37.7676941976696
                        ],
                        [
                            -122.339016420564,
                            37.7658596232049
                        ],
                        [
                            -122.339984096549,
                            37.7658650469385
                        ],
                        [
                            -122.339967791239,
                            37.7676996217596
                        ],
                        [
                            -122.339000091352,
                            37.7676941976696
                        ]
                    ]
                ],
                "type": "Polygon"
            },
            "id": "361a4

## Aggregating results

Each page of results comes as one feature collection. We can combine the features from different pages of results into one big feature collection. Below we will page through all results in the subscription from the past 3 months and make a combined feature collection.

Results in the API are ordered by a `created` timestamp. This corresponds the time that the feature was published to a Feed and does not necessarily match the `observed` timestamp in the feature's properties, which corresponds to when the source imagery for a feature was collected.

In [27]:
latest_feature = subscription_results['features'][0]
creation_datestring = latest_feature['created']
print('latest feature creation date:', creation_datestring)

latest feature creation date: 2022-10-27T22:53:29.202Z


In [28]:
# This date string can be parsed as a datetime and converted to a date
latest_date = parse(creation_datestring).date()
latest_date

datetime.date(2022, 10, 27)

In [29]:
min_date = latest_date - timedelta(days=90)
print('Aggregate all detections from after this date:', min_date)

Aggregate all detections from after this date: 2022-07-29


In [30]:
feature_collection = {'type': 'FeatureCollection', 'features': []}
next_link = subscription_results_url

# Fetch features iteratively until there are no more next links
while next_link:
    results = requests.get(next_link, auth=BASIC_AUTH).json()
    next_features = results['features']

    # Check if there are next features available
    if next_features:
        latest_feature_creation = parse(next_features[0]['created']).date()
        earliest_feature_creation = parse(next_features[-1]['created']).date()
        print('Fetched {} features fetched ({}, {})'.format(
            len(next_features), earliest_feature_creation, latest_feature_creation))
        feature_collection['features'].extend(next_features)
        next_link = get_next_link(results)

    # No next features available
    else:
        next_link = None

print('Total features: {}'.format(len(feature_collection['features'])))

Fetched 250 features fetched (2021-03-11, 2022-10-27)
Fetched 250 features fetched (2019-09-10, 2021-03-11)
Fetched 250 features fetched (2019-07-27, 2019-09-10)
Fetched 250 features fetched (2019-07-27, 2019-07-27)
Fetched 250 features fetched (2019-07-27, 2019-07-27)
Fetched 250 features fetched (2019-06-13, 2019-07-27)
Fetched 250 features fetched (2019-05-10, 2019-06-13)
Fetched 250 features fetched (2019-04-27, 2019-05-10)
Fetched 250 features fetched (2019-04-17, 2019-04-27)
Fetched 250 features fetched (2019-04-17, 2019-04-17)
Fetched 250 features fetched (2019-04-17, 2019-04-17)
Fetched 231 features fetched (2019-04-16, 2019-04-17)
Total features: 2981


## Saving Results
We can now save the combined geojson feature collection to a file.

In [31]:
os.makedirs('data', exist_ok=True)
filename = 'data/collection_{}.geojson'.format(SUBSCRIPTION_ID)
with open(filename, 'w') as file:
    json.dump(feature_collection, file)

FileLink(filename)

/Users/em.steuer/repos/notebooks/jupyter-notebooks/analytics/quickstart/data/collection_9db92275-1d89-4d3b-a0b6-68abd2e94142.geojson

After downloading the aggregated geojson file with the file link above, try importing the data into a geojson-compatible tool for visualization and exploration:
- [geojson.io](http://geojson.io/)
- [kepler gl](https://kepler.gl/demo)

The saved geojson file can also be used to make a geopandas dataframe.

In [32]:
gpd.read_file(filename)

,id,_g4_task_instance_id,_object_class_idx,intersects,model_id,model_version,object_area_m2,object_diagonal_m,object_length_m,object_width_m,...,source_asset_type,source_cloud_cover,source_item,source_item_id,source_item_type,xmax_px,xmin_px,ymax_px,ymin_px,geometry
0,3f758abf-a88e-452a-93bf-b989df5e2fb0,taski_01GGDSYKARNMSENEHY2B4W04H2,1,False,01DPSEHT77T7HHJZHAHCKFDPEK,2019-10-16 00:58:07+00:00,28725.010137,277.002673,252.580333,113.726234,...,ortho_visual,0.0,20190317_184156_63_105d,20190317_184156_63_105d,PSScene,3769,3685,2806,2768,"POLYGON ((-122.32190 37.79856, -122.32191 37.7..."
1,6f896217-3e72-47b9-b65f-dda00551897a,taski_01GGDSYKARNMSENEHY2B4W04H2,1,False,01DPSEHT77T7HHJZHAHCKFDPEK,2019-10-16 00:58:07+00:00,37839.690285,315.165080,286.062241,132.277822,...,ortho_visual,0.0,20190317_184156_63_105d,20190317_184156_63_105d,PSScene,3892,3796,2845,2801,"POLYGON ((-122.31775 37.79765, -122.31776 37.7..."
2,41486086-8f60-4242-ae8a-715a1c7dd2c5,taski_01GGDSYKARNMSENEHY2B4W04H2,1,False,01DPSEHT77T7HHJZHAHCKFDPEK,2019-10-16 00:58:07+00:00,67785.308549,376.562368,302.790302,223.868823,...,ortho_visual,0.0,20190317_184156_63_105d,20190317_184156_63_105d,PSScene,3303,3202,2467,2393,"POLYGON ((-122.33770 37.80879, -122.33772 37.8..."
3,a5e7c66d-ab93-46a8-a5d9-4ae80c174707,taski_01GGDSYKARNMSENEHY2B4W04H2,1,False,01DPSEHT77T7HHJZHAHCKFDPEK,2019-10-16 00:58:07+00:00,20990.256823,294.866162,285.557127,73.506331,...,ortho_visual,0.0,20190317_184156_63_105d,20190317_184156_63_105d,PSScene,4127,4031,2014,1989,"POLYGON ((-122.30954 37.81954, -122.30954 37.8..."
4,0ba91a62-3174-4926-a693-78a5d7b0b6dc,taski_01GGDSYKARNMSENEHY2B4W04H2,1,False,01DPSEHT77T7HHJZHAHCKFDPEK,2019-10-16 00:58:07+00:00,17826.436962,208.570526,184.968781,96.375382,...,ortho_visual,0.0,20190317_184156_63_105d,20190317_184156_63_105d,PSScene,3347,3315,4194,4133,"POLYGON ((-122.33662 37.76174, -122.33664 37.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2976,86a84bc9-61a9-430a-8ec8-51cda2867309,taski_01D8M83TAF00T4216FMNSEY244,1,None,01D3FJCDHEZ1JS56150B7YR8VT,2019-03-29 21:01:06+00:00,16964.645640,292.182788,286.102796,59.295630,...,visual,0.7,20190408_183125_1027,20190408_183125_1027,PSScene3Band,6651,6555,2072,2052,"POLYGON ((-122.30958 37.81943, -122.30958 37.8..."
2977,15374c67-7264-4e4d-b9ae-1918cfda6191,taski_01D8M83TAF00T4216FMNSEY244,1,None,01D3FJCDHEZ1JS56150B7YR8VT,2019-03-29 21:01:06+00:00,3788.069094,128.292116,124.640250,30.392021,...,visual,0.7,20190408_183125_1027,20190408_183125_1027,PSScene3Band,6974,6932,3860,3850,"POLYGON ((-122.29903 37.77078, -122.29903 37.7..."
2978,a7924c83-80ee-4f66-9b8a-927bde558280,taski_01D8M83TAF00T4216FMNSEY244,1,None,01D3FJCDHEZ1JS56150B7YR8VT,2019-03-29 21:01:06+00:00,2820.959943,104.516177,100.691307,28.015924,...,visual,0.7,20190408_183125_1027,20190408_183125_1027,PSScene3Band,6909,6900,3023,2989,"POLYGON ((-122.30100 37.79405, -122.30101 37.7..."
2979,9c058105-a0da-4740-ab10-100802d4ea81,taski_01D8M83TAF00T4216FMNSEY244,1,None,01D3FJCDHEZ1JS56150B7YR8VT,2019-03-29 21:01:06+00:00,3311.149335,133.751098,131.354190,25.207794,...,visual,0.7,20190408_183125_1027,20190408_183125_1027,PSScene3Band,7415,7406,3911,3867,"POLYGON ((-122.28400 37.77021, -122.28401 37.7..."
